[View in Colaboratory](https://colab.research.google.com/github/ayan59dutta/Handwritten-Text-Extractor-using-NN/blob/master/Handwritten_Character_Recognizer_ANN.ipynb)

In [1]:
import time
start = time.time()
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import zipfile
import requests
import io
print('Packages imported in', time.time()-start, 'seconds...')

Packages imported in 0.9553260803222656 seconds...


In [0]:
def download_file_from_google_drive(id='1JLBWOyxNWDxUnIarBVMdAv460z1ICkuh', destination='nist_by_class.zip'):
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 32768

        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)

download_file_from_google_drive()

In [3]:
start = time.time()
with zipfile.ZipFile('nist_by_class.zip', 'r') as zip_ref:
    zip_ref.extractall()
print('Dataset obtained and unzipped in', time.time()-start, 'seconds...')

Dataset obtained and unzipped in 108.94598364830017 seconds...


In [2]:
start = time.time()
with zipfile.ZipFile(io.BytesIO((requests.get('https://drive.google.com/open?id=1JLBWOyxNWDxUnIarBVMdAv460z1ICkuh')).content), 'r') as zip_ref:
    zip_ref.extractall()
print('Dataset obtained and unzipped in', time.time()-start, 'seconds...')

BadZipFile: ignored

In [0]:
class_info = {}
for i in range(48, 58):
    class_info[i-48] = (hex(i)[2:], i, chr(i))
for i in range(65, 65+26):
    class_info[i-65+10] = (hex(i)[2:], i, chr(i))
for i in range(97, 97+26):
    class_info[i-97+36] = (hex(i)[2:], i, chr(i))

In [5]:
start = time.time()
# data_list = []
data_list = np.array([]).reshape(0, 4097)
total_count = 0

for i in class_info:
    print(class_info[i], '\t', time.time()-start, 'seconds', end='\t')
    path = 'by_class/' + class_info[i][0] + '/train_' + class_info[i][0]
    count = 0
    for img in os.listdir(path):
        im = Image.open(path + '/' + img).convert('L').crop((32, 32, 96, 96))
        im_arr = np.array(im, dtype='float32') / 255.0
        flat_im_arr = im_arr.flatten()
        flat_im_arr_labelled = np.insert(flat_im_arr, 0, i)
#         data_list.append(flat_im_arr_labelled)
        data_list = np.append(data_list, flat_im_arr_labelled.reshape(1, 4097), axis=0)
        count += 1
    print(count, 'samples')
#     !rm -rf 'by_class/' + class_info[i][0]
    total_count += count

!rm -rf 'by_class'
data = np.array(data_list, dtype='float32')

print('Total number of samples =', total_count)
print('Shape of final data =', data.shape)

print('All sample images processed in', time.time()-start, 'seconds...')

('30', 48, '0') 	 0.0003457069396972656 seconds	

KeyboardInterrupt: ignored

In [0]:
start = time.time()
data_shuffled = np.random.shuffle(data)
print('Data shuffled in', time.time()-start, 'seconds...')

Data shuffled in 3140.593427658081 seconds...


In [0]:
X_train = data[:627000, 1:]
Y_train = data[:627000, 0]
X_test = data[627000:, 1:]
Y_test = data[627000:, 0]
del data

In [0]:
model = tf.keras.models.Sequential()
layer = tf.keras.layers.Dense(256, input_shape=[64*64*1])
model.add(layer)
model.add(tf.keras.layers.Dense(192))
model.add(tf.keras.layers.Dense(128))
model.add(tf.keras.layers.Dense(62, activation='softmax'))

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               1048832   
_________________________________________________________________
dense_1 (Dense)              (None, 192)               49344     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               24704     
_________________________________________________________________
dense_3 (Dense)              (None, 62)                7998      
Total params: 1,130,878
Trainable params: 1,130,878
Non-trainable params: 0
_________________________________________________________________


In [0]:
Y_train = tf.keras.utils.to_categorical(Y_train)

In [0]:
# X_train = X_train.astype('float32') / 255.0

In [0]:
model.fit(X_train, Y_train)

Epoch 1/1
627000/627000 [==============================] - 2217s 4ms/step - loss: 15.3387


In [0]:
to_predict = X_test[0]

pred = model.predict(X_test[0].reshape(4096,))
print(class_info[np.argmax(pred)][-1])

plt.axis('off')
plt.imshow(to_predict.reshape(64, 64))
plt.show()

ValueError: Error when checking input: expected dense_input to have shape (4096,) but got array with shape (1,)

In [0]:
X_test[0].shape

(4096,)

In [0]:
to_predict.shape

(4096,)

In [0]:
pred

NameError: name 'pred' is not defined